In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
import re
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("../../data/naver_shopping.txt", sep = '\t', encoding = 'utf-8', header = None)

In [26]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords.append('배송')

In [15]:
df.isnull().sum()

0    0
1    0
dtype: int64

In [16]:
df.columns = ['rating', 'text']

In [8]:
len(df['text'])

200000

In [21]:
def rating_to_label(rating):
    if rating >=4 :
        return 1
    else:
        return 0

df['y'] = df['rating'].apply(lambda x: rating_to_label(x))

In [24]:
tagger = Okt()
line = tagger.pos(df['text'][1])
line 

[('택배', 'Noun'),
 ('가', 'Josa'),
 ('엉망', 'Noun'),
 ('이네', 'Josa'),
 ('용', 'Noun'),
 ('저희', 'Modifier'),
 ('집', 'Noun'),
 ('밑', 'Noun'),
 ('에', 'Josa'),
 ('층', 'Noun'),
 ('에', 'Josa'),
 ('말', 'Noun'),
 ('도', 'Josa'),
 ('없이', 'Adverb'),
 ('놔두고가고', 'Verb')]

In [27]:
def text_preprocessed(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]')
    result = hangul.sub('', text)
    okt = Okt()
    Okt_morphs = okt.pos(result)
    words = []

    for word, pos in Okt_morphs:
        if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':
            if len(word) > 1 and word not in stopwords:
                words.append(word)

    words_str = ' '.join(words)
    return words_str

In [28]:
X_texts = []

for idx, row in df.iterrows():
    tokenized_review = text_preprocessed(df.loc[idx, 'text'])
    X_texts.append(tokenized_review)

In [29]:
df['tokenized'] = X_texts

In [38]:
list = df['tokenized'][2]
words_str = ' '.join(list)
words_str  

'아주 바지 정말 구매 가격 대박 바느질 조금 가성 최고'

In [41]:
from tqdm import tqdm
import time

list_to_string = []
for idx, row in tqdm(df.iterrows()):
    list_to_string.append(' '.join(df.loc[idx, 'tokenized']))

list_to_string[:5]

200000it [00:07, 25040.71it/s]


['배공',
 '택배 엉망',
 '아주 바지 정말 구매 가격 대박 바느질 조금 가성 최고',
 '선물 전달 상품 머그컵 당황 바로 누락 확인 바로 선물 큰일 다시 생각',
 '민트 색상 손잡이 도로 사용']

In [42]:
df['tokenized'] = list_to_string
df.head()

,rating,text,y,tokenized
0,5,배공빠르고 굿,1,배공
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,택배 엉망
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1,아주 바지 정말 구매 가격 대박 바느질 조금 가성 최고
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,선물 전달 상품 머그컵 당황 바로 누락 확인 바로 선물 큰일 다시 생각
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,민트 색상 손잡이 도로 사용


In [43]:
df.to_csv('../../data/naver_shopping_tokenized.csv', sep = ",", encoding = 'utf-8')

## DTM 적용

In [49]:
from sklearn.model_selection import train_test_split
y = df['y'].values.tolist()
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['tokenized'], y, test_size = 0.2, random_state = 0)

In [50]:
def tokenizer(text):
    return text.split()

In [51]:
vect = CountVectorizer(min_df = 3, ngram_range=(1, 2))
X_train_tf = vect.fit_transform(X_train_texts)
X_test_tf = vect.transform(X_test_texts)

In [52]:
vocablist = [word for word, number in sorted(vect.vocabulary_.items(), key=lambda x:x[1])]  

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(random_state = 0)
lr.fit(X_train_tf, y_train)

y_pred = lr.predict(X_test_tf)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [54]:
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.78
Precision : 0.742
Recall : 0.850
F1 : 0.792


## TF-IDF 적용

In [55]:
y = df['y']
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['tokenized'], y, test_size = 0.2, random_state = 0)

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.9)
tfidf_vectorizer.fit(X_train_texts)
tfidf_matrix_train = tfidf_vectorizer.transform(X_train_texts)


In [61]:
lr_tfidf = LogisticRegression(random_state = 0)
lr_tfidf.fit(tfidf_matrix_train, y_train)

tfidf_matrix_test = tfidf_vectorizer.transform(X_test_texts)
y_pred = lr_tfidf.predict(tfidf_matrix_test)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [62]:
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.78
Precision : 0.750
Recall : 0.835
F1 : 0.790


## DTM 기준으로 하이퍼파라미터 튜닝 전 성능 평가
* accuracy: 0.78
* Precision : 0.742
* Recall : 0.850
* F1 : 0.792

## TF-IDF 기준으로 하이퍼파라미터 튜닝 전 성능 평가
* accuracy: 0.78
* Precision : 0.750
* Recall : 0.835
* F1 : 0.790